In [1]:
import re
import numpy as np
import copy

In [2]:
def readEvidance(file, graph):
    with open(file) as f:
        ev = [int(x) for x in f.readlines()[0].strip().split()]
    if ev[0]==0:
        return None
    for evidance_index in range(ev[0]):
        var = ev[(2*evidance_index)+1]
        val = ev[(2*evidance_index)+2]
        for func in graph.CPT.keys():
            if var in func:
                var_index = func.index(var)
                new_size = 1
                for v_index in range(len(func)):
                    if not v_index == var_index:
                        new_size *= graph.cardianlities[func[v_index]] 
                new_CPT = np.zeros(new_size)
                stride = 1
                for v_index in range(len(func)-2,var_index,-1):                    
                    stride *= graph.cardianlities[func[v_index+1]]
                
                new_CPT_point = 0
                old_CPT = graph.CPT[func]
                start_point = stride*val
                end_point = len(graph.CPT[func])
                step = stride*graph.cardianlities[func[var_index]]
                
                for i in range(start_point,end_point,step):
                    new_CPT[new_CPT_point:new_CPT_point+stride+1] = old_CPT[i:i+stride+1]
                    new_CPT_point+=stride
                
                graph.CPT[func] = new_CPT
#                 print(var,val,func,old_CPT,new_CPT)
        graph.cardianlities[var]=1
            
    

In [3]:
def nextIndex(lines, index=-1):
    # to handell blank lines or comments
    index+=1
    while index< len(lines):
        content = lines[index].strip().split()
        if len(content)==0 or content[0]=='c': 
            index+=1
        else: 
            return index
    if index >= len(lines):
        print("Reached end of file")
        return None

In [4]:
def readfile(file):
    with open(file) as f:
        lines = f.readlines()    

    index = nextIndex(lines, -1)
        
    if lines[index].strip() =='MARKOV':
        # storing variable_count 
        markov = Graph() 
        index = nextIndex(lines, index)
        
        markov.variable_count = int(lines[index].strip())

        # storing cardinality 
        index = nextIndex(lines, index)
        markov.cardianlities = [int(x) for x in lines[index].strip().split()]
        
        # storing function count
        index = nextIndex(lines, index)
        markov.function_count = int(lines[index].strip())
        
        for function_index in range(markov.function_count):
            index = nextIndex(lines, index)
            assert index,"invalid file format"
            clique = [int(x) for x in lines[index].strip().split()]
            markov.add(clique[1:])
        
        
        for function_index in range(markov.function_count):
            index = nextIndex(lines, index)
            assert index,"invalid file format"
            count = int(lines[index].strip())
            index = nextIndex(lines, index)
            assert index,"invalid file format"
            clique = tuple(markov.functions[function_index])
#             clique = '-'.join(str(x) for x in clique)
            
            markov.CPT[clique]= np.asarray([float(x) for x in lines[index].strip().split()])
            
        return markov
    else:
        print(lines[index].strip()+" yet to write")
    

In [5]:
func = (0,1)

order = (0,1,2)
state = (1,0,0)

cpt = [4.481689, 14.481689, 21.      , 40.481689]
cpt_cardinality = [2,2,2]

index = 0 
pre_card = 0

func_len = len(func)

for f_index in range(len(func)):
    f_order = order.index(func[len(func) - f_index - 1])
    f_state = state[f_order]
    index+= pre_card**f_index * f_state 
    pre_card = cpt_cardinality[f_order]
cpt[index] , index

(21.0, 2)

In [16]:
class Graph(object):
    def __init__(self):
        self.variable_count = 0
        self.cardianlities = []
        self.variables = []
        self.node = {}
        self.CPT = {}
        self.functions = []
        self.function_count = 0
        self.VE_order = []
        self.bucket = []
    
#     def copy_node(self):
#         return copy.copy(self.node)

    def __isEmpty__(self):
        if self.variables == []:
            return True
        return False
    
    def min_degree(self):
        VE_Order = []
        node = copy.copy(self.node)
        while not self.__isEmpty__():
            min_var = None
            min_val = float('inf')
            for v in self.variables:
                v_count = len(node[v].intersection(self.variables))
                if v_count < min_val:
                    min_val = v_count
                    min_var = v
            VE_Order.append(min_var)
            self.remove_node_var(min_var, node)
        self.VE_order = VE_Order
        return VE_Order
    
    def add(self,var): #var is list of variables forming a clique 
        self.functions.append(var)
        for i in range(len(var)):
            if var[i] in self.variables:
                self.node[var[i]] = self.node[var[i]].union(var[0:i]+var[i+1:])
            else:
                self.variables.append(var[i])
#                 print(var,var[:i]+var[i+1:])
                self.node[var[i]] = set(var[0:i]+var[i+1:])
    
    def remove_node_var(self, var, node):
        if self.__isEmpty__():
            return False
        else:
            if var in self.variables:
                self.variables.remove(var)
                for adj_node in node[var]:
                    node[adj_node] = node[var].union(node[adj_node])-{adj_node}
            else:
                print(var+" does not exists")
                return False
            return True

    def add_to_bucket(self, func):
        for v in self.VE_order:
            if v in func:
                self.bucket[v].append(func)
                break
    
    def remove_var_functions(self, var):
        var_functions = [] 
        if len(self.functions) > 0:
            for func in self.functions:
                if var in func:
                        var_functions.append(func)
        for func in var_functions:
            self.functions.remove(func)
        self.function_count = len(self.functions)
        
    def get_fuctions_with_var(self, var):
        var_functions = [] 
        if len(self.functions) > 0:
            for i in range(len(self.functions)):
                if var in self.functions[i]:
                    new_clique = tuple(self.functions[i])
#                     new_clique = "-".join(str(x) for x in self.functions[i])
                    var_functions.append(new_clique)
            if len(var_functions)>0:
                return var_functions
        print("No function for the variable", var)
        return False
    
    def create_bucket_functions(self):
        self.bucket = [[] for _ in range(self.variable_count)]
        for func in self.CPT.keys():
            self.add_to_bucket(func)
    
    def get_new_clique(self,var, func):
        new_clique = set({})
        for f in func:
            new_clique = new_clique.union(set(f))
        new_clique -= {var} 
        new_clique = list(new_clique)
        list(new_clique).sort()
        new_clique_cardinality = [self.cardianlities[x] for x in new_clique]
        new_clique = tuple(new_clique)
        return new_clique,new_clique_cardinality
    
    
    def get_value(self,func , state):#
        # state is a tuple which has the order of the new vars and value assigned and cardinalities
        # func = (0,1)
        # state.order = (0,1,2)
        # state.value = (1,0,0)
        # cpt = [4.481689, 14.481689, 21.      , 40.481689]
        cpt = self.CPT[func]
        # state.cardinality = [2,2,2]
        index = 0 
        pre_card = 0
        func_len = len(func)
        for f_index in range(len(func)):
            f_order = state["order"]index(func[len(func) - f_index - 1])
            index += pre_card**f_index * state["value"][f_order] 
            pre_card = state["cardinality"][f_order]    
        return cpt[index]

    def sum_product(self, var):
        if not self.bucket:
            self.create_bucket_functions()
        var_functions = self.bucket[var]
        assert var_functions,"Invalid VE Order"
        new_clique,new_clique_cardinality = self.get_new_clique(var,var_functions)
        
        new_clique_cardinality_size = 1 
        for i in new_clique_cardinality:
            new_clique_cardinality_size*=i 
        clique_CPT = np.ones((self.cardianlities[var], new_clique_cardinality_size))
        
        temp_clique = list(new_clique)
        temp_clique.append(var)
        temp_clique_cardinalities = []
        for i in temp_clique:
            temp_clique_cardinalities.append(self.cardianlities[i])
        
        state = {}
        state["order"] = temp_clique
        state["cardinality"] = temp_clique_cardinalities
        state["value"] = [0]*len(temp_clique) 
        for i in range(state["cardinality"][-1]):
            for j in range(new_clique_cardinality_size):
                state["value"] = np.zeros(temp_clique) 
                index = temp_clique
                while j>=0:
                    if index// state["value"][index]:
                        state["value"][index]=0
                        index +=1
                    state["value"][index]+=1
                    j-=1
                for func in var_functions:
                    print(self.get_value(func , state))
                    
                    
                
    def do_VE(self):
        for min_var in self.VE_order:
            new_clique, new_clique_CPT = self.sum_product(var)
            if new_clique in self.CPT.keys():
                old_clique_CPT = CPT[new_clique]
                new_clique_CPT *= old_clique_CPT
            CPT[new_clique] = new_clique_CPT
            

SyntaxError: invalid syntax (<ipython-input-16-44329562b163>, line 118)

In [13]:
mar = readfile('1.uai')
readEvidance(file="1.uai.evid", graph=mar)
mar.min_degree()
mar.create_bucket_functions()

In [14]:
mar.bucket

[[(0,), (0, 1), (0, 3)],
 [(1,), (1, 4)],
 [(2,), (1, 2), (2, 5)],
 [(3,), (3, 4)],
 [(4,), (4, 5), (4, 7)],
 [(5,)],
 [(6,), (6, 7), (3, 6)],
 [(7,)],
 [(8,), (7, 8), (5, 8)]]

In [15]:
mar.sum_product(0)

AttributeError: 'dict' object has no attribute 'value'

In [10]:
# %debug

(1, 0, 0)

In [121]:
mar.CPT

{(0,): array([0.87269, 1.1459 ]),
 (1,): array([1.0696 , 0.93492]),
 (2,): array([0.50955, 1.9625 ]),
 (3,): array([1.3823 , 0.72345]),
 (4,): array([2.0507 , 0.48763]),
 (5,): array([0.83542, 1.197  ]),
 (6,): array([0.40954, 2.4418 ]),
 (7,): array([2.0889 , 0.47872]),
 (8,): array([0.49618, 2.0154 ]),
 (0, 1): array([3.6808e+02, 2.7168e-03, 2.7168e-03, 3.6808e+02]),
 (1, 2): array([9.3843e+01, 1.0656e-02, 1.0656e-02, 9.3843e+01]),
 (3, 4): array([1.5148e+03, 6.6016e-04, 6.6016e-04, 1.5148e+03]),
 (4, 5): array([8.0239e+02, 1.2463e-03, 1.2463e-03, 8.0239e+02]),
 (6, 7): array([0.39677, 2.5203 , 2.5203 , 0.39677]),
 (7, 8): array([0.24393, 4.0996 , 4.0996 , 0.24393]),
 (0, 3): array([6.7306e-04, 1.4858e+03, 1.4858e+03, 6.7306e-04]),
 (3, 6): array([1.5296e-02, 6.5377e+01, 6.5377e+01, 1.5296e-02]),
 (1, 4): array([10.748   ,  0.093037,  0.093037, 10.748   ]),
 (4, 7): array([1.8837e-03, 5.3087e+02, 5.3087e+02, 1.8837e-03]),
 (2, 5): array([8.2845e-03, 1.2071e+02, 1.2071e+02, 8.2845e-03

In [17]:
a = set()
a.union([1,2])


{1, 2}

In [25]:
', '.join(str(x) for x in a)

'1, 2, 3, 4, 5'

In [93]:
a=np.ones((2,4))
a[0][:2]*=5

In [94]:
a

array([[5., 5., 1., 1.],
       [1., 1., 1., 1.]])

In [88]:
a[0][2]

1.0

In [95]:
x for x in "4-5".split("-")

SyntaxError: invalid syntax (<ipython-input-95-b292106a30eb>, line 1)

In [46]:
[1,2,3].index(3)

2